# Preparation

Imports libraries

Loads RedMetrics data

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import re
import json
import datetime

from random import randint
from ipywidgets import FloatProgress,IntProgress,IntText,Text,interact,interactive,IntSlider,FloatSlider
from IPython.display import display
from itertools import chain

In [ ]:
pd.__version__

In [ ]:
np.__version__

In [ ]:
processRMDF1522 = not ('rmdf1522' in globals())
processRMDFTest = not ('rmrdftest' in globals())

### common variables

In [ ]:
dataFilesNamesStem = "2018-05-23"
versionSuffix = '-1.52.2'
dataFolderPath = "../../data/"
csvEncoding = 'utf-8'
csvSuffix = '.csv'

checkpointsStem = "tutorial1.Checkpoint"

# Version 1.52.2

In [ ]:
if processRMDF1522 or processRMDFTest:
    # date string to pd.Timestamp
    #  RedMetrics timestamps are always UTC according to doc
    #  https://github.com/CyberCRI/RedMetrics/blob/master/API.md
    dateparse = lambda x: pd.to_datetime(x, utc=True)

if processRMDF1522:
    ## Try loading the pre-processed dataframe
    processedRMDF1522Path = dataFolderPath + dataFilesNamesStem + versionSuffix + "-processed" + csvSuffix
    testUsers1522Path     = dataFolderPath + dataFilesNamesStem + versionSuffix + "-testUsers" + csvSuffix
    processRMDF1522 = False
    try:
        rmdf1522 = pd.read_csv(\
                            processedRMDF1522Path,\
                            dtype=str, parse_dates=['serverTime','userTime'],\
                            date_parser=dateparse,\
                           )
        if rmdf1522.columns[0] == 'Unnamed: 0':
            rmdf1522 = rmdf1522.iloc[:,1:]
        testUsers = pd.read_csv(testUsers1522Path, dtype=str).loc[:,['userId']]
        print("rmdf1522 read_csv success")
    except FileNotFoundError:
        print("rmdf1522 read_csv failed: now loading, processing, saving rmdf1522")
        processRMDF1522 = True

### columns

In [ ]:
if processRMDF1522 or processRMDFTest:
    # In RedMetrics data, 'playerId' is actually a session ID.
    # Permanent player IDs are stored as 'localplayerguid' in 'customdata' attached to 'start' events.
    minimalInitializationColumns = ['customData.localplayerguid']
    initializationColumns = ['customData.localplayerguid', 'playerId', 'type', 'serverTime', 'customData.platform']
    relevantColumns = ['sessionId', 'serverTime', 'userId', 'customData.platform']
    playerFilteringColumns = ['sessionId', 'userId', 'customData.platform', 'serverTime']
    checkpointsRelevantColumns = ['sessionId', 'userId', 'type', 'section']

### dataframes for the different game versions

In [ ]:
if processRMDF1522 or processRMDFTest:
    def getNormalizedRedMetricsCSV( df ):
        newColumns = np.unique(np.concatenate((minimalInitializationColumns, df.columns.values)))
        return safeGetNormalizedRedMetricsCSV(df.loc[:,newColumns])

    def safeGetNormalizedRedMetricsCSV( df ):
        return df.rename(index=str, columns={'customData.localplayerguid' : 'userId', 'playerId': 'sessionId'})

    relevantColumns = ['customData.localplayerguid', 'playerId', 'type']
    renamedRelevantColumns = ['userId', 'sessionId', 'type']

# Old versions

rdf100 = pd.read_csv("../../data/1.0.csv")
rdf131 = pd.read_csv("../../data/1.31.csv")
rdf132 = pd.read_csv("../../data/1.32.csv")
rdf133 = pd.read_csv("../../data/1.33.csv")
rdf140 = pd.read_csv("../../data/1.40.csv")
rdf150 = pd.read_csv("../../data/1.50.csv")
rdf151 = pd.read_csv("../../data/1.51.csv")

part100 = rdf100.loc[:,relevantColumns]
part131 = rdf131.loc[:,relevantColumns]
part132 = rdf132.loc[:,relevantColumns]
part133 = rdf133.loc[:,relevantColumns]
part140 = rdf140.loc[:,relevantColumns]
part150 = rdf150.loc[:,relevantColumns]
part151 = rdf151.loc[:,relevantColumns]

# Tests

rdftest = pd.read_csv("../../data/2017-10-11.test.csv")
dftest = getNormalizedRedMetricsCSV(rdftest)

## Filtering

In [ ]:
if processRMDF1522:
    
    def getAllSessions( _rmDF, dropna ):
        _result = _rmDF.loc[:, renamedRelevantColumns]
        _result = _result[_result['type']=='start']
        _result = _result.drop('type', 1)
        if dropna:
            _result = _result.dropna(how='any')
        return _result

    def getTestSessions(_rmDF, _rmTestDF, includeAndroid = True, includeEditor = True, includeTest = True):

        rmDFTestSessions = set()
        rmTestDFTestSessions = set()

        #  - have 'android' or '...editor' as platform
        if(includeAndroid):
            rmDFTestSessions |= set(_rmDF[_rmDF['customData.platform'].isin(['"android"'])]['sessionId'])
        if(includeEditor):
            rmDFTestSessions |= set(_rmDF[_rmDF['customData.platform'].apply(lambda s: str(s).endswith('editor"'))]['sessionId'])
        #print(str(len(rmDFTestSessions)))

        #  - are in the RedMetrics test channel
        if(includeTest):
            rmTestDFTestSessions = set(_rmTestDF['sessionId'])
        #print(str(len(rmTestDFTestSessions)))

        #  - belong to a user who has a session of the type above
        # all the sessions above
        testSessions = rmDFTestSessions | rmTestDFTestSessions

        return testSessions

    # gets sessions which either:
    #  - have 'android' or '...editor' as platform
    #  - are in the RedMetrics test channel
    #  - belong to a user who has a session of the type above
    def getTestUsersSessions(_rmDF, _rmTestDF, includeAndroid = True, includeEditor = True, includeTest = True):

        # tables of association of user-sessions
        rmDFUsersSessions = getAllSessions(_rmDF, False)
        rmTestDFUsersSessions = getAllSessions(_rmTestDF, False)
        userSessions = pd.concat([rmDFUsersSessions,rmTestDFUsersSessions])

        testSessions = getTestSessions(_rmDF, _rmTestDF,
                                       includeAndroid = includeAndroid, includeEditor = includeEditor, includeTest = includeTest)

        # all the users
        rmDFTestUsers = set(rmDFUsersSessions[rmDFUsersSessions['sessionId'].isin(testSessions)]['userId'].dropna())
        rmTestDFTestUsers = set(_rmTestDF['userId'].dropna())
        rmTestDFTestUsers.remove('')
        testUsers = rmDFTestUsers | rmTestDFTestUsers
        # all the sessions which belong to these users
        allTestSessions = set(rmDFUsersSessions[rmDFUsersSessions['userId'].isin(testUsers)]['sessionId'].dropna())

        return (testUsers,allTestSessions)

TOD: get rid of warning
    
    DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
    interactivity=interactivity, compiler=compiler, result=result)

using https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

## Loading
### Data format fixes

In [ ]:
if processRMDF1522 or processRMDFTest:
    
    def userIdConverter(uId):
        sUID = str(uId)
        if(sUID.startswith('n')):# == 'nan' or == 'null'):
            return ''
        else:
            return sUID.replace('"','')

    loadColumnNames = ['id', 'serverTime', 'userTime',\
                       'playerId', 'playerCustomData',\
                       'type', 'coordinates','section',\
                       'customData.biobrick', 'customData.devices',\
                       'customData.slot', 'customData.sound',\
                       'customData','customData.duration',\
                       'customData.nanobot', 'customData.language',\
                       'customData.controls', 'customData.chapter',\
                       'customData.life','customData.source',\
                       'customData.platform','customData.localplayerguid',\
                       'customData.sametab', 'customData.device',\
                       'customData.energy', 'customData.option',\
                       'customData.newtab','customData.dnabit',\
                       'customData.count', 'customData.plasmid',\
                       'customData.total', 'customData.message',\
                       'customData.graphics']

### RMDFTest loading

In [ ]:
def writeTestUsers():
    testUsers.to_csv(testUsers1522Path, encoding=csvEncoding)

In [ ]:
def writeRMDF():
    rmdf1522.to_csv(processedRMDF1522Path, encoding=csvEncoding)

In [ ]:
# necessary variables for RMDFTest loading:
# dataFolderPath
# dataFilesNamesStem
# dateparse
# userIdConverter
# loadColumnNames
# getNormalizedRedMetricsCSV

In [ ]:
if processRMDFTest:
    rmrdftest = pd.read_csv(\
                                dataFolderPath + dataFilesNamesStem + ".test" + csvSuffix,\
                                dtype=str,\
                                parse_dates=['serverTime','userTime'],\
                                date_parser=dateparse,\
                                converters={'customData.localplayerguid':userIdConverter},\
                           )
    rmrdftest = rmrdftest.loc[:,loadColumnNames]
    normalizedRMDFTest = getNormalizedRedMetricsCSV(rmrdftest)


if(processRMDF1522):
    
    rmrdf1522  = pd.read_csv(\
                            dataFolderPath + dataFilesNamesStem + versionSuffix + csvSuffix,\
                            dtype=str,\
                            parse_dates=['serverTime','userTime'],\
                            date_parser=dateparse,\
                            converters={'customData.localplayerguid':userIdConverter},\
                         )
    rmrdf1522 = rmrdf1522.loc[:,loadColumnNames]
    part1522 = rmrdf1522.loc[:,relevantColumns]
    normalizedRMDF1522 = getNormalizedRedMetricsCSV(rmrdf1522)

    (testUsers, allTestSessions) = getTestUsersSessions(_rmDF = normalizedRMDF1522, _rmTestDF = normalizedRMDFTest)
    rmdf1522 = normalizedRMDF1522[~normalizedRMDF1522['sessionId'].isin(allTestSessions)]
    
    testUsers = pd.DataFrame(data=list(testUsers), columns=['userId'])
    writeTestUsers()

    userSessions = rmdf1522[rmdf1522['userId']!=''].loc[:,['userId','sessionId']]
    
    intProgress = IntProgress(min=0, max=len(userSessions.index))
    display(intProgress)
    intText = IntText(0)
    display(intText)
    
    for userSessionsIndex in userSessions.index:
        intProgress.value += 1
        intText.value += 1
        
        userId = userSessions.loc[userSessionsIndex, 'userId']
        sessionId = userSessions.loc[userSessionsIndex, 'sessionId']
        rmdf1522.loc[rmdf1522['sessionId']==sessionId,'userId'] = userId

    rmdf1522['userId'].nunique(),userSessions['userId'].nunique(),\
    rmdf1522[~rmdf1522['userId'].isin(userSessions['userId'].unique())],\
    userSessions[~userSessions['userId'].isin(rmdf1522['userId'].unique())]

#### Saving to csv

In [ ]:
if(processRMDF1522):
    writeRMDF()

# All versions

rdf = pd.concat([part100, 
                      part131, part132, part133, 
                      part140, 
                      part150, part151, part1522])

df = getNormalizedRedMetricsCSV(rdf)